<a href="https://colab.research.google.com/github/Nicolasalan/Object-Detection-Tensorflow/blob/main/TensorFlow2_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalar dependências de detecção de objetos do TensorFlow2**

In [ ]:
!nvidia-smi

Fri Jul 15 04:10:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pathlib

# Clone o repositório de modelos do tensorflow se ele ainda não existir
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3421, done.
remote: Counting objects: 100% (3421/3421), done.
remote: Compressing objects: 100% (2855/2855), done.
remote: Total 3421 (delta 892), reused 1410 (delta 509), pack-reused 0
Receiving objects: 100% (3421/3421), 35.13 MiB | 23.04 MiB/s, done.
Resolving deltas: 100% (892/892), done.


In [ ]:
# Instale a API de detecção de objetos
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
# executa o teste do construtor de modelos
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
def load_image_into_numpy_array(path):
  """Carrega uma imagem do arquivo em uma matriz numpy.

   Coloca a imagem no array numpy para alimentar o gráfico do tensorflow.
   Observe que, por convenção, colocamos em uma matriz numpy com forma
   (altura, largura, canais), onde canais=3 para RGB.

   Args:
     caminho: um caminho de arquivo.

   Devoluções:
     uint8 matriz numpy com forma (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Função wrapper para visualizar detecções.

   Args:
     image_np: array numpy uint8 com forma (img_height, img_width, 3)
     caixas: uma matriz numpy de forma [N, 4]
     classes: uma matriz numpy de forma [N]. Observe que os índices de classe são baseados em 1,
       e combine as chaves no mapa de rótulos.
     pontuações: uma matriz numpy de forma [N] ou Nenhum. Se pontuações=Nenhuma, então
       esta função assume que as caixas a serem plotadas são verdadeiras
       caixas e plote todas as caixas como pretas sem classes ou pontuações.
     category_index: um dict contendo dicionários de categorias (cada um contendo
       índice de categoria `id` e nome da categoria `name`) codificados por índices de categoria.
     figsize: tamanho da figura.
     image_name: um nome para o arquivo de imagem.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
!pip install "opencv-python-headless<4.3"
!pip install 'tensorflow==2.7.0'
!pip install 'tf-models-official==2.7.0' 
!pip install 'tensorflow_io==0.23.1'

# **Dataset import**

In [ ]:
%cd /content/

%%bash

mkdir DataSet
mkdir -p /content/DataSet/test
mkdir -p /content/DataSet/train
cp /content/drive/MyDrive/weight/train.record /content/DataSet/train
cp /content/drive/MyDrive/weight/eval.record /content/DataSet/test
cp /content/drive/MyDrive/weight/Label_Map.txt /content/DataSet/

# **Configuraçōes do DataSet**

**Trocar as extensão `record` para `tfrecord`, para ser compativel com TensorFlow 2.**

In [8]:
%cd /content/DataSet/test/
!mv eval.record test.tfrecord

/content/DataSet/test


In [10]:
%cd /content/DataSet/train/
!mv train.record train.tfrecord

/content/DataSet/train


In [11]:
test_record_fname = '/content/DataSet/test/test.tfrecord' 
train_record_fname = '/content/DataSet/train/train.tfrecord' 
label_map_pbtxt_fname = '/content/DataSet/Label_Map.pbtxt' 

# **Configuração de treinamento de detecção de objetos TensorFlow2**

In [ ]:
## Altere o modelo escolhido para implantar diferentes modelos disponíveis no zoológico de detecção de objetos do TF2

MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 16
    }
}

# O modelo efficientdet-d0 é o mais leve e eficiente do estado da arte
# se você quiser escalar para modelos mais eficientes, provavelmente precisará de mais computação!
chosen_model = 'efficientdet-d0'

num_steps = 40000 # Quanto mais passos, mais longo o treinamento. Aumente se sua função de perda ainda estiver diminuindo e as métricas de validação estiverem aumentando.
num_eval_steps = 500 # Realizar avaliação após tantos passos

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] # se você conseguir colocar um lote grande na memória, isso pode acelerar seu treinamento

In [ ]:
# baixar pesos pré-treinados

%mkdir /content/models/research/deploy/
%cd /content/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

mkdir: cannot create directory ‘/content/models/research/deploy/’: File exists
/content/models/research/deploy
--2022-07-15 09:50:45--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.199.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.199.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz.4’

efficientdet_d0_coc 100%[===================>]  29.31M  --.-KB/s    in 0.1s    

2022-07-15 09:50:45 (217 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz.4’ saved [30736482/30736482]



In [ ]:
# baixa o arquivo de configuração do treinamento básico

%cd /content/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/models/research/deploy
--2022-07-15 09:50:47--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.4’

ssd_efficientdet_d0 100%[===================>]   4.52K  --.-KB/s    in 0s      

2022-07-15 09:50:47 (64.9 MB/s) - ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config.4’ saved [4630/4630]



In [ ]:
pipeline_fname = '/content/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

# caso já tenha treinado uma rede e quer continuar aonde paro, retirar o jogo da velha nessa linha de baixo e comente a de cima
# fine_tune_checkpoint = '{caminho do repositorio}/tuned_model/checkpoint/ckpt-0'
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)


In [ ]:
# escreva o arquivo de configuração personalizado colocando o conjunto de dados, ponto de verificação do modelo e parâmetros de treinamento no arquivo de pipeline base

import re

%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # arquivos tfrecord treinam e testam.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Definir batch_size de treinamento.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Definir etapas de treinamento, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Configura o número de classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # ajustar o tipo de ponto de verificação
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)



/content/models/research/deploy
writing custom configuration file


In [ ]:
%cat /content/models/research/deploy/pipeline_file.config

 # SSD with EfficientNet-b0 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d0).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b0 checkpoint.
#
# Train on TPU-8

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 18
    add_background_class: false
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      

In [ ]:
pipeline_file = '/content/models/research/deploy/pipeline_file.config'
model_dir = '/content/training/'

# **Treinar o Detector de Objetos TF2 Personalizado**

* pipeline_file: definido acima ao escrever a configuração de treinamento personalizada
* model_dir: os logs do tensorboard de localização e os pontos de verificação do modelo salvos serão salvos em
* num_train_steps: por quanto tempo treinar
* num_eval_steps: executa eval no conjunto de validação após tantas etapas

**Adicionar script ao console desse navegador:** `inspecionar` => `console` => `adicionar script`.
```
function ConnectButton(){
    console.log("Conectado"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```







Executar treinamento ...

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=10 \
    --num_eval_steps={num_eval_steps}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/RoboCup-2022/Stage-2/training/train'

## **Exportando um gráfico de inferência treinado**
Plotando os gráficos para verificar o desempenho da rede neural

In [ ]:
# listas pesos treinados
%ls '/content/training/'

checkpoint                  ckpt-3.index
ckpt-1.data-00000-of-00001  ckpt-4.data-00000-of-00001
ckpt-1.index                ckpt-4.index
ckpt-2.data-00000-of-00001  ckpt-5.data-00000-of-00001
ckpt-2.index                ckpt-5.index
ckpt-3.data-00000-of-00001  train/


In [ ]:
# executa script de conversão

import re
import numpy as np

output_directory = '/content/fine_tuned_model'

# exportar os pesos treinados para o arquivo saved_model
last_model_path = '/content/training/'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/content/drive/MyDrive/RoboCup-2022/Stage-2/train/training/
2022-07-15 07:48:05.393440: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0715 07:48:05.400726 140497955256192 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet EfficientNet backbone version: efficientnet-b0
I0715 07:48:05.400952 140497955256192 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 64
I0715 07:48:05.401035 140497955256192 ssd_efficientnet_bifpn_feature_extractor.py:149] EfficientDet BiFPN num iterations: 3
I0715 07:48:05.404447 140497955256192 efficientnet_model.py:147] round_filter input=32 output=32
I0715 07:48:05.426615 140497955256192 efficientnet_model.py:147] round_filter input=32 output=32
I0715 07:48:05.426765 140497955256192 efficientnet_model.py:147] round_filter input=16 output=16
I0715 07:48:05.482882 140497955256192 

In [ ]:
%ls '/content/fine_tuned_model/saved_model/'

# **Exportar modelo**

Exportar modelo para o Google Drive na pasta weight criada no Colab anterior

In [ ]:
!cp -r /content/fine_tuned_model/saved_model /content/drive/MyDrive/weight/
!cp -r /content/DataSet/Label_Map.pbtxt /content/drive/MyDrive/weight/saved_model